In [ ]:
! pip install rdkit-pypi

In [ ]:
from pathlib import Path
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, Draw, PandasTools

In [ ]:
molecules = pd.read_csv("/content/output_file_first_step.csv", index_col=0)
print(molecules.shape)
molecules.head()

In [ ]:
def lipinski(smiles, verbose=False):
    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData= np.arange(1,1)
    i=0
    for mol in moldata:
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])

        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1

    columnNames=["molecular_weight","logp","n_hbd","n_hba"]
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)

    return descriptors
def lipinski_pass(row):
    return row['molecular_weight'] <= 500 and \
           row['logp'] <= 5 and \
           row['n_hbd'] <= 5 and \
           row['n_hba'] <= 10
ro5_properties = lipinski(molecules.smiles)
ro5_properties['passes_ro5'] = ro5_properties.apply(lipinski_pass, axis=1)

In [ ]:
ro5_properties

In [ ]:
molecules

In [ ]:
molecules = pd.concat([molecules, ro5_properties], axis=1)
molecules

In [ ]:
molecules = molecules[molecules['passes_ro5']]
molecules

In [ ]:
molecules = molecules[molecules['class'] != 'intermediate']
molecules

In [ ]:
molecules.to_csv('/content/sample_data/ro5_properties_filtered.csv')
molecules.head(10)

In [ ]:
def calculate_mean_std(dataframe):
    stats = dataframe.describe()
    stats = stats.T
    stats = stats[["mean", "std"]]
    return stats

In [ ]:
molecules_stats = calculate_mean_std(
    molecules[["molecular_weight", "n_hba", "n_hbd", "logp"]]
)
molecules_stats

In [ ]:
def _scale_by_thresholds(stats, thresholds, scaled_threshold):
    for property_name in stats.index:
        if property_name not in thresholds.keys():
            raise KeyError(f"Add property '{property_name}' to scaling variable.")
    stats_scaled = stats.apply(lambda x: x / thresholds[x.name] * scaled_threshold, axis=1)
    return stats_scaled

In [ ]:
def _define_radial_axes_angles(n_axes):
    x_angles = [i / float(n_axes) * 2 * math.pi for i in range(n_axes)]
    x_angles += x_angles[:1]
    return x_angles

In [ ]:
def plot_radar(
    y,
    thresholds,
    scaled_threshold,
    properties_labels,
    y_max=None,
    output_path=None,
):

    x = _define_radial_axes_angles(len(y))
    y = _scale_by_thresholds(y, thresholds, scaled_threshold)
    y = pd.concat([y, y.head(1)])


    plt.figure(figsize=(6, 6))
    ax = plt.subplot(111, polar=True)


    ax.fill(x, [scaled_threshold] * len(x), "cornflowerblue", alpha=0.2)
    ax.plot(x, y["mean"], "b", lw=3, ls="-")
    ax.plot(x, y["mean"] + y["std"], "orange", lw=2, ls="--")
    ax.plot(x, y["mean"] - y["std"], "orange", lw=2, ls="-.")


    ax.set_theta_offset(math.pi / 2)
    ax.set_theta_direction(-1)

    ax.set_rlabel_position(180)
    plt.xticks(x, [])
    if not y_max:
        y_max = int(ax.get_yticks()[-1])
    plt.ylim(0, y_max)
    plt.yticks(
        range(1, y_max),
        ["5" if i == scaled_threshold else "" for i in range(1, y_max)],
        fontsize=16,
    )
    for i, (angle, label) in enumerate(zip(x[:-1], properties_labels)):
        if angle == 0:
            ha = "center"
        elif 0 < angle < math.pi:
            ha = "left"
        elif angle == math.pi:
            ha = "center"
        else:
            ha = "right"
        ax.text(
            x=angle,
            y=y_max + 1,
            s=label,
            size=16,
            horizontalalignment=ha,
            verticalalignment="center",
        )
    labels = ("mean", "mean + std", "mean - std", "rule of five area")
    ax.legend(labels, loc=(1.1, 0.7), labelspacing=0.3, fontsize=16)
    plt.savefig('radar.png', dpi=600, bbox_inches="tight", transparent=True)
    plt.show()

In [ ]:
thresholds = {"molecular_weight": 500, "n_hba": 10, "n_hbd": 5, "logp": 5}
scaled_threshold = 5
properties_labels = [
    "Molecular weight (Da) / 100",
    "# HBA / 2",
    "# HBD",
    "LogP",
]
y_max = 8

In [ ]:
plot_radar(
    molecules_stats,
    thresholds,
    scaled_threshold,
    properties_labels,
    y_max,
)

In [ ]:
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.countplot(x='class', data=molecules, edgecolor='black')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

plt.savefig('plot_bioactivity_class.png', dpi=600, bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("white")

custom_palette = sns.color_palette("Set2")

plt.figure(figsize=(7, 7))

sns.scatterplot(
    x='molecular_weight',
    y='logp',
    data=molecules,
    hue='class',
    size='pIC50',
    sizes=(20, 200),
    palette=custom_palette,
    edgecolor='black',
    alpha=0.7
)

plt.xlabel('Molecular Weight (MW)', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')

plt.grid(False)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, title='Legend')
sns.despine()
plt.savefig('plot_MW_vs_LogP.png', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'molecular_weight', data = molecules)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.png', dpi=600, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'logp', data = molecules)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')

plt.savefig('plot_LogP.png', dpi=600, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'n_hbd', data = molecules)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHDonors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHDonors.png', dpi=600, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'n_hba', data = molecules)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHAcceptors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHAcceptors.png', dpi=600, bbox_inches='tight')

In [ ]:
! zip -r results.zip . -i *.csv *.pdf